# AIOE 계산 구현

이 노트북에서는 전처리된 데이터를 사용하여 각 직업의 **AI Occupational Exposure (AIOE)** 점수를 계산합니다.

## AIOE란?

AIOE는 특정 직업이 AI 기술(특히 Language Modeling)에 얼마나 노출되어 있는지를 측정하는 지표입니다.
- **높은 AIOE**: AI가 해당 직업의 많은 능력을 대체하거나 보완할 수 있음
- **낮은 AIOE**: AI가 해당 직업의 능력을 대체하기 어려움

## 0. 환경설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

In [ ]:
# 데이터 경로 설정
PROCESSED_DIR = '/content/drive/MyDrive/aioe_project/datas/processed'

## 1. 전처리된 데이터 불러오기

### 이전 단계에서 우리가 만든 데이터

01번 노트북에서 우리는 2개의 데이터를 합쳐서 하나의 파일을 만들었습니다:
- **O*NET Abilities**: 894개 직업 × 52개 능력의 Importance, Level
- **MTurk 매핑**: 52개 능력의 Beta 값 (AI 영향도)

**최종 데이터 구조:**
- **46,488개 행** (894 직업 × 52 능력)
- **6개 컬럼**: SOC_Code, Occupation_Title, Ability, Importance, Level, Beta

**이제 할 일:**
이 데이터를 사용해서 각 직업의 **AIOE 점수를 계산**합니다.
- 입력: 46,488개 행 (직업-능력 조합)
- 출력: 894개 행 (직업별 AIOE 점수)

In [ ]:
# 전처리된 데이터 로드
df = pd.read_csv(f'{PROCESSED_DIR}/preprocessed_data.csv')

print(f"✅ 데이터 로드 완료: {df.shape[0]:,}행 × {df.shape[1]}컬럼, {df['soc_code'].nunique()}개 직업")

✅ 데이터 로드 완료: 46,488행 × 7컬럼, 894개 직업


## 2. AIOE 계산 아이디어

### 우리가 해결하려는 문제

**질문:** "내 직업이 AI에 얼마나 노출되어 있을까?"

**가지고 있는 정보:**
1. 특정 직업에 필요한 52개 능력 (Importance, Level)
2. AI가 각 능력을 얼마나 잘하는지 (Beta)

### 계산 아이디어

**핵심 질문:**
> "전체 능력 중에서 AI가 도와줄 수 있는 능력이 차지하는 비중은?"

**실제 Software Developers 데이터 5개로 살펴봅시다:**

| 능력 (한국어) | Importance | Level | Scalar<br/>(= I × L) | Beta | 가중 영향도<br/>(= Scalar × Beta) |
|--------------|-----------|-------|---------|------|------------------|
| 듣고 이해하기<br/>(Oral Comprehension) | 3.75 | 4.12 | **15.45** | 0.906 | 13.998 |
| 글 읽고 이해하기<br/>(Written Comprehension) | 3.75 | 4.38 | **16.425** | 0.827 | 13.584 |
| 말로 표현하기<br/>(Oral Expression) | 3.75 | 4.12 | **15.45** | 0.906 | 13.998 |
| 글로 표현하기<br/>(Written Expression) | 3.50 | 4.00 | **14.00** | 0.848 | 11.872 |
| 아이디어 떠올리기<br/>(Fluency of Ideas) | 3.25 | 3.62 | **11.765** | 0.717 | 8.436 |
| **5개 능력 합계** | - | - | **73.09** | - | **61.888** |

**컬럼 의미:**
1. **Importance**: 이 능력이 직업에서 얼마나 중요한가 (1-5 척도)
2. **Level**: 이 능력이 얼마나 높은 수준을 요구하는가 (0-7 척도)
3. **Scalar** (I × L): 능력의 가중치 (중요도 × 수준)
4. **Beta**: AI가 이 능력을 얼마나 잘하는가 (0-1, MTurk 데이터)
5. **가중 영향도** (Scalar × Beta): AI가 이 능력에서 도와줄 수 있는 정도

**직접 계산해볼까요?**
- 듣고 이해하기 Scalar = 3.75 × 4.12 = **15.45**
- 가중 영향도 = 15.45 × 0.906 = **13.998**

**5개 능력만으로 AIOE를 계산하면:**
- AIOE = 61.888 ÷ 73.09 = **0.847 (84.7%)**
- 이 5개 능력만 보면 AI가 84.7% 도와줄 수 있다는 의미

### 계산 순서 (5단계)

우리는 이제 **단계별로** 계산해 나갈 것입니다:

1. **정규화**: 서로 다른 척도를 0-1로 통일
2. **Scalar 계산**: 정규화된 값으로 능력 가중치 계산
3. **가중 영향도 계산**: 각 능력에서 AI의 기여도
4. **직업별 집계**: 52개 능력을 모두 합산
5. **표준화**: 다른 직업과 비교 가능하도록 변환

**지금은 복잡해 보이지만, 한 단계씩 같이 해볼게요!**

---

## 3. Step 1: Importance와 Level을 0-1 범위로 변환하기

### 왜 변환이 필요한가요?

**현재 문제:**
- Importance: 1~5 척도
- Level: 1~7 척도
→ **서로 다른 척도를 곱하면 공정하지 않음!**

**해결책: 정규화 (Normalization)**
- Importance: 원래 값 ÷ 5  (예: 3.5 → 0.70)
- Level: 원래 값 ÷ 7  (예: 3.88 → 0.55)

**결과:**
- 모든 값이 0~1 사이로 변환
- 공정한 비교 가능
- 0 = 최소, 1 = 최대

## 4. Step 2: Scalar 계산하기 (정규화 후)

### 이제 정규화된 값으로 Scalar를 계산합니다

**Scalar = scale_level × scale_importance**

**정규화 전 vs 후 비교:**

| 능력 | 정규화 전<br/>I × L | 정규화 후<br/>I × L |
|------|-----------------|-----------------|
| Written Comprehension | 3.75 × 4.38 = **16.425** | 0.75 × 0.626 = **0.469** |
| Fluency of Ideas | 3.25 × 3.62 = **11.765** | 0.65 × 0.517 = **0.338** |


**결론:**
- **Scalar가 클수록** = 그 직업의 핵심 능력
- **Scalar가 작을수록** = 덜 중요한 능력

## 5. Step 3: AI 가중 영향도 계산하기

### 가중 영향도란?

**가중 영향도 = Scalar × Beta**

**의미:**
- Scalar: 이 능력이 직업에서 얼마나 중요한가? (능력의 가중치)
- Beta: 이 능력을 AI가 얼마나 잘하는가? (AI 영향도)
- **가중 영향도**: AI가 **실제로** 이 직업에 미치는 영향

**핵심 아이디어:**
```
능력이 중요해도 AI가 못하면 → 영향 없음
AI가 잘해도 능력이 안 중요하면 → 영향 없음
둘 다 높아야 → 큰 영향!
```

**구체적인 상황:**

| 능력 | Scalar | Beta | 가중 영향도 | 해석 |
|------|----------|---------|-----------|------|
| Written Comprehension | 0.60 | 0.83 | **0.50** | 🔥 AI가 큰 영향 |
| Manual Dexterity | 0.08 | 0.20 | **0.02** | ✅ AI 영향 없음 |

---

## 6. Step 4: 직업별 집계 및 AIOE 기본 점수 계산

### 이제 모든 조각을 합칠 시간입니다!

**지금까지 계산한 것:**
- ✅ Scalar (각 능력의 가중치)
- ✅ 가중 영향도 (AI가 각 능력에서 도와줄 수 있는 비중)

**이제 할 일:**
- 52개 능력을 모두 합산해서 직업별 AIOE 계산

### AIOE 공식

$$
AIOE_j = \frac{\sum_{a=1}^{52} (L_{ja} \times I_{ja} \times \beta_a)}{\sum_{a=1}^{52} (L_{ja} \times I_{ja})}
$$

**공식 분해:**
- $\sum_{a=1}^{52} (L_{ja} \times I_{ja} \times \beta_a)$ = **분자**: 52개 능력의 가중 영향도 합 (AI가 도와줄 수 있는 총 비중)
- $\sum_{a=1}^{52} (L_{ja} \times I_{ja})$ = **분모**: 52개 능력의 Scalar 합 (전체 능력 가중치 = 100%)
- **결과**: AI가 도와줄 수 있는 비중 / 전체 비중 = AI 도움 비율

**쉽게 말하면:**
```python
AIOE = sum(가중 영향도) / sum(Scalar)
     = AI가 도와줄 수 있는 총 비중 / 전체 능력 가중치
     = AI가 도와줄 수 있는 비율
```

In [ ]:
# 직업별 집계 및 AIOE 점수 계산
# groupby(): 같은 직업 끼리 그룹으로 묶음
# agg(): 각 그룹의 값들을 합산 또는 개수 세기
# reset_index(): 그룹화된 컬럼을 다시 일반 컬럼으로 변환


---

## 7. Step 5: 표준 점수(Z-score)로 변환하기

### 왜 Z-score로 변환하나요?

**현재 문제:**
- AIOE 기본 점수는 **절대값** (예: 0.685)
- 이 숫자만 봐서는 "높은건가 낮은건가?" 판단하기 어려움

**해결책: Z-score 변환**
- 각 점수를 **상대적 위치**로 변환
- "다른 직업들과 비교해서 어느 정도인가?"를 보여줌

### 3명 학생의 시험 점수를 생각해봅시다

**점수:**
- 철수: 70점
- 영희: 85점  
- 민수: 90점
- **평균**: 81.67점

**절대 점수만 보면:**
- 철수가 얼마나 뒤처졌는지 애매함

**Z-score로 변환하면:**
- 철수: -1.2 (평균보다 낮음)
- 영희: +0.3 (평균보다 약간 높음)
- 민수: +0.9 (평균보다 높음)

→ **상대적 위치가 명확해집니다!**

### Z-score 공식

**Z-score = (내 점수 - 평균) / 표준편차**

**해석:**
- **Z = 0**: 평균 (중간)
- **Z > 0**: 평균보다 높음
- **Z < 0**: 평균보다 낮음
- **Z > 1**: 상위 16% (높음)
- **Z > 2**: 상위 2.5% (매우 높음)

## 8. 결과 저장

In [ ]:
# 마스터 테이블 저장
master_file = f'{PROCESSED_DIR}/aioe_master.csv'
results_df.to_csv(master_file, index=False)

---

## 9. 최종 요약

### 이번 노트북에서 배운 것

**AIOE 계산 과정 (5단계):**

1. **정규화** - Importance(1-5), Level(0-7)을 0-1 범위로 변환
2. **Scalar 계산** - Level × Importance (능력의 가중치)
3. **가중 영향도 계산** - Scalar × Beta (AI의 실제 영향도)
4. **직업별 집계** - 52개 능력을 합산하여 직업당 1개 점수 산출
5. **Z-score 변환** - 상대적 위치를 알 수 있도록 표준화

### 핵심 공식

$$
AIOE_j = \frac{\sum_{a=1}^{52} (Level_{ja} \times Importance_{ja} \times Beta_a)}{\sum_{a=1}^{52} (Level_{ja} \times Importance_{ja})}
$$

**해석:** 전체 능력 중 AI가 도와줄 수 있는 능력의 비율

### 결과 데이터

**저장 위치:** `../datas/processed/aioe_scores.csv`

**데이터 구조:**
- 894개 직업 × 7개 컬럼
- 주요 컬럼: `SOC_Code`, `Occupation_Title`, `ability_base_impact`, `aioe`, `rank`

**AIOE 범위:**
- 최고: +1.79 (Z-score)
- 최저: -1.97 (Z-score)
- 평균: 0.00 (Z-score 특성)

### 다음 단계

**03_AIOE_EDA.ipynb**에서:
- AIOE 점수 분포 시각화
- 직업군별 비교 분석
- Software Developers 사례 분석
- 상관관계 탐색